# Importaciones y funciones necesarias

In [73]:
#importamos la librerias necesarias
from numpy import nan
import pandas as pd
import ast

In [74]:
#creamos una funcion para buscar el indice name 
def buscar_nombre(lista):
    #en este caso esto convierte los objetos a listas
    lista = ast.literal_eval(lista)
    # en caso de haber mas de un diccionario con la clave name devuelve todos los valores concatenados
    return ', '.join([i['name'] for i in lista])

#creamos otra funcion con la misma logica  que buscar nombre pero modificada para funcionar con diccionarios
def buscar_nombre_diccionario(dicc):
    #en este caso esto convierte los objetos a diccionarios
    dicc = ast.literal_eval(dicc)
    return dicc['name']

#creamos una funcion para buscar el indice name si el job es igual a director
def buscar_nombre_director(lista):
    #en este caso esto convierte los objetos a listas
    lista = ast.literal_eval(lista)
    for i in lista:
            if i['job'] == 'Director':
                return i['name']
            else: continue


# ETL de movies_dataset.csv

In [75]:
#leemos el csv
movies = pd.read_csv('data/movies_dataset.csv')
#comprobamos que el dataframe este cargado correctamente
movies

C:\Users\Miguel\AppData\Local\Temp\ipykernel_8980\1581738893.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  movies = pd.read_csv('data/movies_dataset.csv')


adult                              belongs_to_collection    budget  \
0      False  {'id': 10194, 'name': 'Toy Story Collection', ...  30000000   
1      False                                                NaN  65000000   
2      False  {'id': 119050, 'name': 'Grumpy Old Men Collect...         0   
3      False                                                NaN  16000000   
4      False  {'id': 96871, 'name': 'Father of the Bride Col...         0   
...      ...                                                ...       ...   
45461  False                                                NaN         0   
45462  False                                                NaN         0   
45463  False                                                NaN         0   
45464  False                                                NaN         0   
45465  False                                                NaN         0   

                                                  genres  \
0      [{'id': 16, 'name': 'Animation'}, {'id': 35, '...   
1      [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...   
2      [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...   
3      [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...   
4                         [{'id': 35, 'name': 'Comedy'}]   
...                                                  ...   
45461  [{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...   
45462                      [{'id': 18, 'name': 'Drama'}]   
45463  [{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...   
45464                                                 []   
45465                                                 []   

                                   homepage      id    imdb_id  \
0      http://toystory.disney.com/toy-story     862  tt0114709   
1                                       NaN    8844  tt0113497   
2                                       NaN   15602  tt0113228   
3                                       NaN   31357  tt0114885   
4                                       NaN   11862  tt0113041   
...                                     ...     ...        ...   
45461  http://www.imdb.com/title/tt6209470/  439050  tt6209470   
45462                                   NaN  111109  tt2028550   
45463                                   NaN   67758  tt0303758   
45464                                   NaN  227506  tt0008536   
45465                                   NaN  461257  tt6980792   

      original_language               original_title  \
0                    en                    Toy Story   
1                    en                      Jumanji   
2                    en             Grumpier Old Men   
3                    en            Waiting to Exhale   
4                    en  Father of the Bride Part II   
...                 ...                          ...   
45461                fa                      رگ خواب   
45462                tl          Siglo ng Pagluluwal   
45463                en                     Betrayal   
45464                en          Satana likuyushchiy   
45465                en                     Queerama   

                                                overview  ... release_date  \
0      Led by Woody, Andy's toys live happily in his ...  ...   1995-10-30   
1      When siblings Judy and Peter discover an encha...  ...   1995-12-15   
2      A family wedding reignites the ancient feud be...  ...   1995-12-22   
3      Cheated on, mistreated and stepped on, the wom...  ...   1995-12-22   
4      Just when George Banks has recovered from his ...  ...   1995-02-10   
...                                                  ...  ...          ...   
45461        Rising and falling between a man and woman.  ...          NaN   
45462  An artist struggles to finish his work while a...  ...   2011-11-17   
45463  When one of her hits goes wrong, a professiona...  ...   2003-08-01   
45464  In a small town live two brothers, one a minis...  ...   1917-10-21   
45465  50 years after decriminalisati

In [76]:
#comprobamos el tamaño del dataframe
print(movies.shape)
#removemos los valores duplicados
movies.drop_duplicates(inplace=True)
print(movies.shape)
#removemos las columnas que no se utilizaran y/o estan duplicadas
movies = movies.drop(columns=['video', 'adult', 'imdb_id', 'original_title', 'poster_path', 'homepage', 'overview','tagline'])
print(movies.shape)


(45466, 24)
(45453, 24)
(45453, 16)


In [77]:
#remplazamos los nulos por 0
movies['revenue'] = movies['revenue'].fillna('0')
movies['budget'] = movies['budget'].fillna('0')

In [78]:
#removemos las filas con valores faltantes en la columna release_date
movies = movies.dropna(subset=['release_date'])
#eliminamos las filas que no sigan el formato AAAA-MM-DD en la columna release_date
movies['release_date'] = pd.to_datetime(movies['release_date'], format='%Y-%m-%d', errors='coerce')
movies = movies.dropna(subset=['release_date'])

In [79]:
#creamos la columna year 
movies['released_year'] = movies['release_date'].dt.year

In [80]:
#cambiamos el tipo de las columnas budget y revenue a int para poder hacer operaciones
movies['budget'] = movies['budget'].astype(int)
movies['revenue'] = movies['revenue'].astype(int)
#creamos la columna return que es la columna revenue entre la columna budget
    #en caso de que uno de los numeros sea 0 devolvera 0
movies['return'] = movies.apply(lambda fila: fila['revenue'] / fila['budget'] if fila['budget'] != 0 else 0, axis=1)


In [81]:
#aplicamos la funcion buscar_nombre a las columnas para desanidar los datos 
movies['genres'] = movies['genres'].fillna('[]').apply(buscar_nombre)
movies['spoken_languages'] = movies['spoken_languages'].fillna('[]').apply(buscar_nombre)
movies['production_companies'] = movies['production_companies'].fillna('[]').apply(buscar_nombre)
movies['production_countries'] = movies['production_countries'].fillna('[]').apply(buscar_nombre)

In [82]:
#en caso de no haber datos en belongs_to_collection la funcion fillna crea un diccionario con la clave name y el valor "NA"
    #de no hacerlo daria un key error al ejecutar la funcion buscar_nombre_diccionario
movies['belongs_to_collection'] = movies['belongs_to_collection'].fillna('{"name":"NA"}').apply(buscar_nombre_diccionario)

movies['belongs_to_collection'] = movies['belongs_to_collection'].replace('NA', nan)

In [83]:
movies

belongs_to_collection    budget                      genres  \
0                Toy Story Collection  30000000   Animation, Comedy, Family   
1                                 NaN  65000000  Adventure, Fantasy, Family   
2           Grumpy Old Men Collection         0             Romance, Comedy   
3                                 NaN  16000000      Comedy, Drama, Romance   
4      Father of the Bride Collection         0                      Comedy   
...                               ...       ...                         ...   
45460                             NaN         0      Drama, Action, Romance   
45462                             NaN         0                       Drama   
45463                             NaN         0     Action, Drama, Thriller   
45464                             NaN         0                               
45465                             NaN         0                               

           id original_language popularity  \
0         862                en  21.946943   
1        8844                en  17.015539   
2       15602                en    11.7129   
3       31357                en   3.859495   
4       11862                en   8.387519   
...       ...               ...        ...   
45460   30840                en   5.683753   
45462  111109                tl   0.178241   
45463   67758                en   0.903007   
45464  227506                en   0.003503   
45465  461257                en   0.163015   

                                    production_companies  \
0                                Pixar Animation Studios   
1      TriStar Pictures, Teitler Film, Interscope Com...   
2                           Warner Bros., Lancaster Gate   
3                 Twentieth Century Fox Film Corporation   
4             Sandollar Productions, Touchstone Pictures   
...                                                  ...   
45460  Westdeutscher Rundfunk (WDR), Working Title Fi...   
45462                                        Sine Olivia   
45463                            American World Pictures   
45464                                          Yermoliev   
45465                                                      

                                    production_countries release_date  \
0                               United States of America   1995-10-30   
1                               United States of America   1995-12-15   
2                               United States of America   1995-12-22   
3                               United States of America   1995-12-22   
4                               United States of America   1995-02-10   
...                                                  ...          ...   
45460  Canada, Germany, United Kingdom, United States...   1991-05-13   
45462                                        Philippines   2011-11-17   
45463                           United States of America   2003-08-01   
45464                                             Russia   1917-10-21   
45465                                     United Kingdom   2017-06-09   

         revenue  runtime   spoken_languages    status  \
0      373554033     81.0            English  Released   
1      262797249    104.0  English, Français  Released   
2              0    101.0            English  Released   
3       81452156    127.0            English  Released   
4       76578911    106.0            English  Released   
...          ...      ...                ...       ...   
45460          0    104.0            English  Released   
45462          0    360.0                     Released   
45463          0     90.0            English  Released   
45464          0     87.0                     Released   
45465          0     75.0            English  Released   

                             title  vote_average  vote_count  released_year  \
0                        Toy Story           7.7      5415.0           1995   
1                          Jumanji           6.9      2413.0 

# etl de credits.csv

In [84]:
#leemos el csv
credits = pd.read_csv('credits.csv')
#comprobamos que el dataset se haya cargado correctamente
credits

FileNotFoundError: [Errno 2] No such file or directory: 'credits.csv'

In [ ]:
#comprobamos el tamaño del dataframe
print(credits.shape)
# eliminamos los duplicados y volvemos a comprobar el tamaño del dataframe
credits.drop_duplicates(inplace=True)
print(credits.shape)

(45476, 3)
(45439, 3)


In [ ]:
# obtenemos los nombres de los actores
credits['cast'] = credits['cast'].fillna('[]').apply(buscar_nombre)

In [ ]:
#obtenemos los nombres de los directores
credits['director'] = credits['crew'].apply(buscar_nombre_director)

In [ ]:
#eliminamos la columna crew para reducir significativamente el peso del dataframe
credits = credits.drop(columns='crew')

In [ ]:
credits

cast      id  \
0      Tom Hanks, Tim Allen, Don Rickles, Jim Varney,...     862   
1      Robin Williams, Jonathan Hyde, Kirsten Dunst, ...    8844   
2      Walter Matthau, Jack Lemmon, Ann-Margret, Soph...   15602   
3      Whitney Houston, Angela Bassett, Loretta Devin...   31357   
4      Steve Martin, Diane Keaton, Martin Short, Kimb...   11862   
...                                                  ...     ...   
45471          Leila Hatami, Kourosh Tahami, Elham Korda  439050   
45472  Angel Aquino, Perry Dizon, Hazel Orencio, Joel...  111109   
45473  Erika Eleniak, Adam Baldwin, Julie du Page, Ja...   67758   
45474  Iwan Mosschuchin, Nathalie Lissenko, Pavel Pav...  227506   
45475                                                     461257   

               director  
0         John Lasseter  
1          Joe Johnston  
2         Howard Deutch  
3       Forest Whitaker  
4         Charles Shyer  
...                 ...  
45471  Hamid Nematollah  
45472          Lav Diaz  
45473    Mark L. Lester  
45474  Yakov Protazanov  
45475     Daisy Asquith  

[45439 rows x 3 columns]

# concatenamos y exportamos como csv

In [ ]:
movies = pd.concat([movies, credits], axis=1, join='inner')

movies_limpio_path = 'data/movies_limpio.csv'

movies.to_csv(movies_limpio_path, sep=',', index=False, encoding='utf-8')
